This notebook describes how to generate predictions of labels for a determined dataset, from a model containing the neural network weights.

# Load necessary modules

In [1]:
# show images inline
%matplotlib inline

# automatically reload modules when they have changed
%load_ext autoreload
%autoreload 2

# import keras
import keras

# import keras_retinanet
from keras_retinanet.models.resnet import custom_objects
from keras_retinanet.utils.image import read_image_bgr, preprocess_image, resize_image

# import miscellaneous modules
import matplotlib.pyplot as plt
import cv2
import os
import numpy as np
import time

# set tf backend to allow memory to grow, instead of claiming everything
import tensorflow as tf

def get_session():
    config = tf.ConfigProto()
    config.gpu_options.allow_growth = True
    return tf.Session(config=config)

# use this environment flag to change which GPU to use
os.environ["CUDA_VISIBLE_DEVICES"] = "1"

# set the modified tf session as backend in keras
keras.backend.tensorflow_backend.set_session(get_session())

import pandas as pd

/home/barimpac/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


## Load RetinaNet model

In [3]:
# adjust this to point to your downloaded/trained model
model_path = os.path.join('/home/barimpac/keras-retinanet/snapshots/old_2', 'resnet50_csv_50.h5')

# load retinanet model
model = keras.models.load_model(model_path, custom_objects=custom_objects)
#print(model.summary())

# load label to names mapping for visualization purposes
labels_to_names = {0: 'defect'}

/home/barimpac/anaconda3/lib/python3.6/site-packages/keras/models.py:274: UserWarning: Output "nms" missing from loss dictionary. We assume this was done on purpose, and we will not be expecting any data to be passed to "nms" during training.
  sample_weight_mode=sample_weight_mode)


# Store image names and predictions

In [17]:
# Path of the directory containing the images that you would like to label.
# Make sure you have only images in this directory"
d = '/home/barimpac/keras-retinanet/data/imgs/test/'

images = os.listdir(d)

In [18]:
labels = []

for image in images:
    # load image
    img_name = image
    image = read_image_bgr(d+image)
    
# preprocess image for network
    image = preprocess_image(image)
    image, scale = resize_image(image)

# process image
    start = time.time()
    _, _, detections = model.predict_on_batch(np.expand_dims(image, axis=0))
    print("processing time: ", time.time() - start)

# compute predicted labels and scores
    predicted_labels = np.argmax(detections[0, :, 4:], axis=1)
#print(predicted_labels)
    scores = detections[0, np.arange(detections.shape[1]), 4 + predicted_labels]
#print(scores)

# correct for image scale
    detections[0, :, :4] /= scale
#print(detections)

# visualize detections
    for idx, (label, score) in enumerate(zip(predicted_labels, scores)):
        if score < 0.5:
            #print("t")
            continue
        b = detections[0, idx, :4].astype(int)
        labels.append([img_name, b])

processing time:  1.8735449314117432
processing time:  1.2867965698242188
processing time:  1.4992785453796387
processing time:  0.9653220176696777
processing time:  1.328805923461914
processing time:  1.5543527603149414
processing time:  1.188965082168579
processing time:  1.4571928977966309
processing time:  1.6101233959197998
processing time:  1.1563506126403809
processing time:  1.414827823638916
processing time:  1.5007376670837402
processing time:  1.1251506805419922
processing time:  1.6065592765808105
processing time:  1.450901746749878
processing time:  1.1128466129302979
processing time:  1.646498441696167
processing time:  1.3393757343292236
processing time:  1.2754936218261719
processing time:  1.5909039974212646
processing time:  1.367602825164795
processing time:  1.3895187377929688
processing time:  1.6230723857879639
processing time:  1.1576652526855469
processing time:  1.4360978603363037
processing time:  1.672267198562622
processing time:  1.079819917678833
processin

In [19]:
df = pd.DataFrame(labels, columns=['image', 'defect coordinates'])

In [20]:
# name of the file to save the predictions
df.to_csv("test1.csv")